# Modelo usando Sklearn

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
import pandas as pd
import numpy as np

# Modelo para los intents de cocina y varios con Sklearn

In [53]:
dataset = pd.read_csv('intents_cocina_varios.csv')
dataset.tail()

,Unnamed: 0,text,label
1201,1201,Hola que como estas,varios
1202,1202,Buenos dias,varios
1203,1203,Quien eres,varios
1204,1204,Como te llamas,varios
1205,1205,Cual es tu nombre,varios


In [54]:
sentences = dataset['text'].values
y = dataset['label'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(
    sentences, y, test_size=0.01, random_state=1000)

vectorizer = CountVectorizer(min_df=0, lowercase=False) 
vectorizer.fit(sentences_train) 

CountVectorizer(lowercase=False, min_df=0)

In [55]:
x = vectorizer.transform(sentences_train)

In [56]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(x, y_train) 

LogisticRegression()

In [57]:
text = vectorizer.transform(["Como puedo preparar"])
classifier.predict(text)

array(['cocina'], dtype=object)

# Modelo usando redes neuronales

In [173]:
input_dim = x.shape[0]
model = Sequential()
model.add(Dense(10, input_dim=input_dim, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [174]:
model.compile(loss="binary_crossentropy",
             optimizer='adam',
             metrics=['accurancy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 10)                11940     
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 11        
Total params: 11,951
Trainable params: 11,951
Non-trainable params: 0
_________________________________________________________________


In [175]:
n = np.array([0]*1193)
x.shape

(1193, 1750)

In [176]:
history = model.fit(x, n,
                   epochs=40,
                   verbose=False,
                   batch_size=10)

ValueError: in user code:

    /home/david/bin/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /home/david/bin/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/david/bin/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/david/bin/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/david/bin/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/david/bin/anaconda3/lib/python3.8/site-pack

# Modelo para los intents de cocina

In [61]:
df = pd.read_csv('intents_cocina.csv')
df.tail()

,Unnamed: 0,text,label
6354,6354,"Si tengo 1 1/4 tazas de azúcar, 1 taza de agua...",title
6355,6355,Quiero Sorbete de mora como se hace,title_instructions
6356,6356,Quiero Rollos de la Casa Parker como se hace,title_instructions
6357,6357,Como preparar Orzo con jamón y queso de cabra,title_instructions
6358,6358,Quiero Alcachofas al vapor con mantequilla de ...,title_instructions


In [62]:
sentences = df['text'].values
y = df['label'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(
    sentences, y, test_size=0.20, random_state=1000)

vectorizer = CountVectorizer(min_df=0, lowercase=False) 
vectorizer.fit(sentences_train) 
x_train = vectorizer.transform(sentences_train)
x_test = vectorizer.transform(sentences_test)

In [72]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(x_train, y_train) 

score = classifier.score(x_test, y_test)
print('{:.4f}'.format(score))

1.0000


/home/david/bin/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [51]:
text = vectorizer.transform(["Que hago con "])
classifier.predict(text)

array(['title_instructions'], dtype=object)